# En este notebook vamos a aplicar algunas herrammientas aprendidas para hacer clasificación. Para ello utilizaremos la Mammographic Dataset, que se encuentra en http://archive.ics.uci.edu/ml/index.php

1. Title: Mammographic Mass Data

2. Sources:

   (a) Original owners of database:
        Prof. Dr. R�diger Schulz-Wendtland
        Institute of Radiology, Gynaecological Radiology, University Erlangen-Nuremberg
        Universit�tsstra�e 21-23
        91054 Erlangen, Germany
        
   (b) Donor of database:
        Matthias Elter
        Fraunhofer Institute for Integrated Circuits (IIS)
        Image Processing and Medical Engineering Department (BMT) 
        Am Wolfsmantel 33
        91058 Erlangen, Germany
        matthias.elter@iis.fraunhofer.de
        (49) 9131-7767327 
        
   (c) Date received: October 2007
 
3. Past Usage:
    M. Elter, R. Schulz-Wendtland and T. Wittenberg (2007)
    The prediction of breast cancer biopsy outcomes using two CAD approaches that both emphasize an intelligible decision process.
    Medical Physics 34(11), pp. 4164-4172

4. Relevant Information:
    Mammography is the most effective method for breast cancer screening
    available today. However, the low positive predictive value of breast
    biopsy resulting from mammogram interpretation leads to approximately
    70% unnecessary biopsies with benign outcomes. To reduce the high
    number of unnecessary breast biopsies, several computer-aided diagnosis
    (CAD) systems have been proposed in the last years.These systems
    help physicians in their decision to perform a breast biopsy on a suspicious
    lesion seen in a mammogram or to perform a short term follow-up
    examination instead.
    This data set can be used to predict the severity (benign or malignant)
    of a mammographic mass lesion from BI-RADS attributes and the patient's age.
    It contains a BI-RADS assessment, the patient's age and three BI-RADS attributes
    together with the ground truth (the severity field) for 516 benign and
    445 malignant masses that have been identified on full field digital mammograms
    collected at the Institute of Radiology of the
    University Erlangen-Nuremberg between 2003 and 2006.
    Each instance has an associated BI-RADS assessment ranging from 1 (definitely benign)
    to 5 (highly suggestive of malignancy) assigned in a double-review process by
    physicians. Assuming that all cases with BI-RADS assessments greater or equal
    a given value (varying from 1 to 5), are malignant and the other cases benign,
    sensitivities and associated specificities can be calculated. These can be an
    indication of how well a CAD system performs compared to the radiologists.

5. Number of Instances: 961

6. Number of Attributes: 6 (1 goal field, 1 non-predictive, 4 predictive attributes)

7. Attribute Information:
   1. BI-RADS assessment: 1 to 5 (ordinal)  
   2. Age: patient's age in years (integer)
   3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
   4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
   5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
   6. Severity: benign=0 or malignant=1 (binominal)

8. Missing Attribute Values: Yes
    - BI-RADS assessment:    2
    - Age:                   5
    - Shape:                31
    - Margin:               48
    - Density:              76
    - Severity:              0

9. Class Distribution: benign: 516; malignant: 445

In [ ]:
# Cargamos las librerías esenciales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/mammographic-masses/mammographic_masses.data"
mam_data=pd.read_csv(data_url, na_values='?', header=-1)

In [ ]:
# Veamos info de los datos
mam_data.info()

In [ ]:
# Y su estadística más descriptiva
mam_data.describe()

In [ ]:
# Las primeras líneas
mam_data.head()

In [ ]:
# Redefinamos el nombre de las columnas, modificando el atributo columns
mam_data.columns=['bi_rads', 'age','shape','margin','density','severity']

In [ ]:
# Ploteemos los datos en histogramas, por ejemplo, para ver las 
# diferencias entre grupos con diferente severidad
fig, axs=plt.subplots(ncols=2, nrows=3)
axs=axs.flatten()
for i,ax in enumerate(axs):
    ax.hist(mam_data.dropna()[mam_data.dropna().iloc[:,5]==0].iloc[:,i])
    ax.hist(mam_data.dropna()[mam_data.dropna().iloc[:,5]==1].iloc[:,i])

## Plotear categorical y ordinal data usando seaborn

Existe una librería muy conocida en python llamada seaborn, que se basa en matplotlib pero que a diferencia de ésta, trabaja nativamente con dataframes, permitiendo hacer plots más avanzados en menos líneas. No es el objetivo de este curso saber usar esta librería. Quien esté interesado, puede visitar la página oficial https://seaborn.pydata.org/ y mirar diferentes tutoriales que se pueden encontrar por internet.

In [ ]:
import seaborn as sns
for var in ['bi_rads','shape','margin','density']:
    plt.figure()
    sns.countplot(x=var, hue='severity', data=mam_data)

In [ ]:
sns.violinplot(x='severity', y='age', data=mam_data)

## Sin NANs

Como hemos podido comprobar, la dataset contienen NaNs. En scikit, introducir una matriz de features o vector de labels con NaNs daría error. Por ello, tenemos que tratarlos de alguna forma. El método más sencillo sería desechar aquellas observaciones (filas de la matriz de features) que contengan NaNs. Esto se puede hacer fácilmente mediante el método **dropna** de los dataframes de pandas 

In [ ]:
mam_data_clean = mam_data.dropna()

In [ ]:
print(mam_data_clean.info())

Creamos como siempre una matriz de features y un vector de targets o labels

In [ ]:
X=mam_data_clean.drop(columns=['bi_rads','severity']).values
y=mam_data_clean['severity'].values

Partimos los datos en train (70%) y test (30%) usando la funcion `train_test_split de scikit`, que se encuentra en el módulo `model_selection` (Veremos con detenimiento dicho módulo la semana que viene)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=0)

Tenemos dos variables categóricas que tenemos que codificar y tratar correctamente. Ya hemos visto cómo hacer esto en scikit mediante el módulo `preprocessing`

In [ ]:
from sklearn.preprocessing import OneHotEncoder
oHe=OneHotEncoder(categorical_features=[1,2], sparse=False)
X_train_ohe = oHe.fit_transform(X_train) 
X_test_ohe = oHe.transform(X_test) # Notad aquí que uso transform!

In [ ]:
# Mirad la diferencia en el número de columnas de los datos 
# transformados y sin transformar

print("el numero de columnas originales = ", X_train.shape[1])
print("el numero de columnas despues de transformar los datos = ", 
      X_train_ohe.shape[1])

Vamos a probar un árbol de decisión y random forest, que no requieren mucho preprocessing de los datos

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

clf=DecisionTreeClassifier(random_state=0)
clf.fit(X_train_ohe, y_train)
print("la accuracy del arbol de decision es: ", 
      clf.score(X_test_ohe, y_test))

clf=RandomForestClassifier(random_state=0)
clf.fit(X_train_ohe, y_train)
print("la accuracy del random forest es: ", 
      clf.score(X_test_ohe, y_test))

In [ ]:
for depth in np.arange(1,10):
    clf=DecisionTreeClassifier(max_depth=depth, random_state=0)
    clf.fit(X_train_ohe, y_train)
    plt.plot(depth, clf.score(X_train_ohe, y_train), '.b')    
    plt.plot(depth, clf.score(X_test_ohe, y_test), '.r')
    plt.ylabel('accuracy')
    plt.xlabel('depth')

Usemos ahora otro clasificador, por ejemplo Support Vector Machines. 

In [ ]:
from sklearn.svm import SVC
clf=SVC(random_state=0)

Este algoritmo requiere que las variables tengan la misma escala. Usemos por ejemplo la clase `MinMaxScaler` para llevar a acabo esto

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_ohe_scaler =  scaler.fit_transform(X_train_ohe)
X_test_ohe_scaler =  scaler.transform(X_test_ohe)

In [ ]:
print(X_train_ohe.max(axis=0))
print(X_train_ohe_scaler.max(axis=0))

In [ ]:
clf.fit(X_train_ohe_scaler, y_train)
print("la accuracy de SVM es: ", clf.score(X_test_ohe_scaler, y_test))

Podemos ver cómo varía la clasificación cambiado los parámetros del clasificador

In [ ]:
for C in 10**np.arange(-2,2, 0.2):
    clf=SVC(C=C, random_state=0)
    clf.fit(X_train_ohe_scaler, y_train)
    plt.plot(C, clf.score(X_test_ohe_scaler, y_test), '.r') # Mejor en escala logarítmica
    #plt.semilogx(C, clf.score(X_test_ohe_scaler, y_test), '.r')
    plt.ylabel('accuracy')
    plt.xlabel('C')

Lo mismo para KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

for neigh in np.arange(1,20, 1):
    clf=KNeighborsClassifier(n_neighbors=neigh)
    clf.fit(X_train_ohe_scaler, y_train)
    plt.plot(neigh, clf.score(X_test_ohe_scaler, y_test), '.r')
    plt.ylabel('accuracy')
    plt.xlabel('neighbors')

Otra medida importante del rendimiento de un clasificador suele ser el área bajo la curva ROC, que te dice cómo varía el clasificador cuando se cambia el treshold

In [ ]:
#importamos las clases que vamos a usar
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
#Fiteamos por ejemplo un support vector machines
clf=SVC(random_state=0, probability=True)
clf.fit(X_train_ohe_scaler, y_train)
print(clf.score(X_test_ohe_scaler, y_test))
y_pred_probs = clf.predict_proba(X_test_ohe_scaler)

In [ ]:
y_pred_probs.shape

In [ ]:
# Calculamos la tasa de true y false positive a medida que 
# varíamos la probabilidad 
fpr, tpr, _ = roc_curve(y_test, y_pred_probs[:,1])

In [ ]:
# Calculamos el áera bajo la curva
auc = roc_auc_score(y_test, y_pred_probs[:,1])

In [ ]:
plt.plot(fpr, tpr, label= "AUC = {:f}".format(auc))
plt.legend()

### De todas formas, ya hemos visto que podemos tratar los NaNs mediante la clase `Imputer` en el módulo `preprocessing`. Veamos cómo lo aplicaríamos sobre estos datos

In [ ]:
from sklearn.preprocessing import Imputer

In [ ]:
X = mam_data.drop(columns=['bi_rads', 'severity']).values
y = mam_data['severity'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=0)

In [ ]:
#Imputar por la media para la variable edad
imp_age = Imputer(strategy='mean')
X_train[:,0] = imp_age.fit_transform(X_train[:,0].reshape(-1,1)).flatten()
X_test[:,0] = imp_age.transform(X_test[:,0].reshape(-1,1)).flatten() # Notad aquí sólo transform

In [ ]:
#imputar por el valor más frequente (útil para variables categóricas 
# y ordinales)
imp_cat = Imputer(strategy='most_frequent')
X_train[:,1:] = imp_cat.fit_transform(X_train[:,1:])
X_test[:,1:] = imp_cat.transform(X_test[:,1:]) #Notad aquí sólo transform

In [ ]:
X_train_ohe = oHe.fit_transform(X_train)
X_test_ohe = oHe.transform(X_test) # Notad aquí sólo transform
scaler = MinMaxScaler()
X_train_ohe_scaler =  scaler.fit_transform(X_train_ohe)
X_test_ohe_scaler =  scaler.transform(X_test_ohe) # Notad aquí sólo transform

In [ ]:
for C in 10**np.arange(-2,2, 0.2):
    clf=SVC(C=C, random_state=0)
    clf.fit(X_train_ohe_scaler, y_train)
    plt.plot(C, clf.score(X_test_ohe_scaler, y_test), '.r') # Mejor en escala logarítmica
    plt.semilogx(C, clf.score(X_test_ohe_scaler, y_test), '.r')
    plt.ylabel('accuracy')
    plt.xlabel('C')

Hemos hecho varias operaciones en este notebook: dividir en train y test, one Hot encoding + MinMaxScaler + Imputer para tratar los NaNs. Después hemos ajustado y predicho. Muchas de estas operaciones se tienen que calcular sólo sobre el training set. No podemos usar el test set para tomar decisiones sobre qué features coger, qué factor de normalización tomar para reescalar los datos, etc... Realizar tantas operaciones sobre los datos implica más probabilidad de equivocarnos. Veremos las próxima semanas que scikit tiene un módulo muy útil (ciertamente es uno de sus puntos fuertes) llamado `pipeline`, que permite encadenar muchos pasos a realizar sobre los datos en una sola línea. Esto nos llevará a economizar en ejecuciones, pero también a saber que todo lo que encadenemos se realiza en la parte de los datos adecuados.